<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60">

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'>


Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego 
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej".   
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# Statistical machine learning - Notebook 8, version for students
**Author: Michał Ciach**  
**Date: 30.11.2021**

## Description
 

In today's class, we will learn how to perform linear regression.  
It's a method of fitting a linear function to a data set with a single predicted numerical variable $Y$ and several explanatory variables (a.k.a. predictors or independent variables) stored in a matrix $X$.  
The model of linear regression can be written as  
$$Y_i = X_i\beta + \epsilon_i,$$
where $X_i$ is the i-th row of the $X$ matrix and $\epsilon_i$ is the effect of factors influencing the value of $Y$ that are not included in $X$. This effect is called an *error*, but note that it's a slightly misleading name.   

When we want to include categorical variables as predictors (i.e. model the differences of income between voivodeships), we create so-called *dummy variables*: for each category of the variable, we create a binary dummy variable which is equal to 1 if a given observation comes from this category and 0 otherwise. We will see examples of dummy variables in the first exercise.  

In [1]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1GW1pjKOCoKOlC4Jqbqql_ghYD_n0iC6O
!gdown https://drive.google.com/uc?id=1FInZ2jrlZGNColU4sHF9JKGHP39fTVut
!gdown https://drive.google.com/uc?id=1n1qS6dcVVKcVJOuUIIm0VTz6cSyrtzDH

Downloading...
From: https://drive.google.com/uc?id=1GW1pjKOCoKOlC4Jqbqql_ghYD_n0iC6O
To: /content/BDL municipality incomes 2015-2020.csv
100% 228k/228k [00:00<00:00, 32.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1FInZ2jrlZGNColU4sHF9JKGHP39fTVut
To: /content/BDL municipality area km2 2015-2020.csv
100% 180k/180k [00:00<00:00, 25.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1n1qS6dcVVKcVJOuUIIm0VTz6cSyrtzDH
To: /content/BDL municipality population 2015-2020.csv
100% 222k/222k [00:00<00:00, 68.4MB/s]


## Data & library imports

In this notebook, we'll introduce another Python library for statistical data analysis. The `statsmodels` library implements several statistical tests and methods for linear regression.  

In [2]:
import pandas as pd
import plotly.express as px
import numpy as np
import statsmodels.api as sm
from scipy.linalg import svd

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [3]:
income = pd.read_csv('BDL municipality incomes 2015-2020.csv', sep=';', dtype={'Code': 'str'})
population = pd.read_csv('BDL municipality population 2015-2020.csv', sep='\t', dtype={'Code': 'str'})
area = pd.read_csv('BDL municipality area km2 2015-2020.csv', sep='\t', dtype={'Code': 'str'})

In [4]:
voivodeship_names = {
    '02': 'Dolnośląskie',
    '04': 'Kujawsko-pomorskie',
    '06': 'Lubelskie',
    '08': 'Lubuskie',
    '10': 'Łódzkie',
    '12': 'Małopolskie',
    '14': 'Mazowieckie',
    '16': 'Opolskie',
    '18': 'Podkarpackie',
    '20': 'Podlaskie',
    '22': 'Pomorskie',
    '24': 'Śląskie',
    '26': 'Świętokrzyskie',
    '28': 'Warmińsko-mazurskie',
    '30': 'Wielkopolskie',
    '32': 'Zachodniopomorskie'
}

In [5]:
code_list = [s[:2] for s in income["Code"]]
name_list = [voivodeship_names[code] for code in code_list]
income['Voivodeship'] = name_list

## Linear regression and SVD

The estimator of $\beta$ is given by the equation
$$\hat{\beta} = (X^TX)^{-1}X^TY.$$
From a computational point of view, using this equation is inefficient and can lead to numerical errors.  
A more efficient approach is based on the [Singular Value Decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition) of the $X$ matrix, given by $X = U\Sigma V^T$.  
We have already used the SVD decomposition to implement the Principal Component Analysis in one of the previous notebooks.   

**Exercise 1.** In this exercise, we will implement a linear regression method using the SVD decomposition and create a linear regression model to predict the income of a municipality in 2020 based on its population and voivodeship. 

Use the SVD decomposition of $X$ to obtain a more efficient formula for $\hat{\beta}$.  

Implement a function that takes an numpy vector of dependent variables `Y` and a numpy array of independent variables `X` and, using the SVD decomposition, computes and returns the estimated regression parameters. 

Create a data frame with the territorial code, income, population and voivodeships of municipalities in 2020 by using `pd.merge` to perform a join with the `Code` variable as the key. Remove rows with missing values.   
Use the `pd.get_dummies()` function to encode the voivodeship for each municipality with dummy variables.  
Use your function to fit a linear regression model that explains the income with the population and voivodeship. 

Compare the results of your implementation with the one from `statsmodels`. You can find the relevant documentation [here](https://www.statsmodels.org/stable/examples/notebooks/generated/ols.html).  
What are the interpretations of the parameters?  
Can you use a model with intercept in this exercise? Why / why not? If yes, what is its interpretation?

In [17]:
## Write your code here.
def regression_parameters_with_svd(Y, X):
  U, S, Vt = np.linalg.svd(X, full_matrices=False)
  x_hat = Vt.T @ np.linalg.inv(np.diag(S)) @ U.T @ Y
  return x_hat

In [9]:
joint = pd.merge(income, population, on=['Code'], suffixes=['_income', '_population'])

In [19]:
data = joint[['Code', '2020_income', '2020_population', 'Voivodeship']]
data = data.dropna()
data

,Code,2020_income,2020_population,Voivodeship
0,0201011,1.138563e+08,38486.0,Dolnośląskie
1,0201022,4.288890e+07,14863.0,Dolnośląskie
2,0201032,2.754443e+07,5317.0,Dolnośląskie
3,0201043,3.341908e+07,15229.0,Dolnośląskie
4,0201052,2.484304e+07,7288.0,Dolnośląskie
...,...,...,...,...
2504,3218043,1.941628e+07,7885.0,Zachodniopomorskie
2505,3218053,2.503944e+07,6856.0,Zachodniopomorskie
2506,3261011,3.840694e+08,106235.0,Zachodniopomorskie
2507,3262011,1.739014e+09,398255.0,Zachodniopomorskie


In [20]:
data = pd.get_dummies(data, columns=['Voivodeship'])
data

,Code,2020_income,2020_population,Voivodeship_Dolnośląskie,Voivodeship_Kujawsko-pomorskie,Voivodeship_Lubelskie,Voivodeship_Lubuskie,Voivodeship_Mazowieckie,Voivodeship_Małopolskie,Voivodeship_Opolskie,Voivodeship_Podkarpackie,Voivodeship_Podlaskie,Voivodeship_Pomorskie,Voivodeship_Warmińsko-mazurskie,Voivodeship_Wielkopolskie,Voivodeship_Zachodniopomorskie,Voivodeship_Łódzkie,Voivodeship_Śląskie,Voivodeship_Świętokrzyskie
0,0201011,1.138563e+08,38486.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0201022,4.288890e+07,14863.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0201032,2.754443e+07,5317.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0201043,3.341908e+07,15229.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0201052,2.484304e+07,7288.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2504,3218043,1.941628e+07,7885.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2505,3218053,2.503944e+07,6856.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2506,3261011,3.840694e+08,106235.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2507,3262011,1.739014e+09,398255.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [21]:
y = data['2020_income']
cols = [col for col in data.columns if col not in ['2020_income', 'Code']]
x = data[cols]

In [22]:
param = regression_parameters_with_svd(y, x)
param

array([ 5.70705853e+03, -3.66339290e+07, -4.21675839e+07, -3.40602587e+07,
       -3.53528117e+07, -2.44553973e+07, -5.12369193e+07, -3.98876731e+07,
       -4.45168036e+07, -2.99246900e+07, -4.27787378e+07, -3.87683584e+07,
       -4.07442405e+07, -3.44689208e+07, -3.56490964e+07, -6.30971674e+07,
       -3.99185736e+07])

In [23]:
model = sm.OLS(y, x)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            2020_income   R-squared:                       0.939
Model:                            OLS   Adj. R-squared:                  0.939
Method:                 Least Squares   F-statistic:                     2381.
Date:                Tue, 11 Jan 2022   Prob (F-statistic):               0.00
Time:                        00:00:11   Log-Likelihood:                -48400.
No. Observations:                2477   AIC:                         9.683e+04
Df Residuals:                    2460   BIC:                         9.693e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
2020_popul

**Exercise 2.** In this exercise, we will perform the *diagnostics* of our model, i.e. we will check if the model and individual variables are significant, if the assumptions of the linear regression are satisfied, and if there are any outlying observations.  

Inspect the test statistics from the `statsmodels` summary of the model fitted in Exercise 1.  
Is the model significant according to the F-test?  
Are all individual variables significant according to the t-test?

Get the fitted values from the `statsmodels` model. You can use the `get_prediction()` function, the `predict()` function, or the `fittedvalues` attribute of the fitted model.      
Use them to compute the residuals $\hat{\epsilon}_i = Y_i - X_i\hat{\beta}$.   
Calculate the standardized residuals $(\hat{\epsilon}_i - \text{mean}(\hat{\epsilon}))/\text{sd}(\hat{\epsilon})$.

Calculate the [coefficient of determination](https://en.wikipedia.org/wiki/Coefficient_of_determination), one of the most common measures of the goodness-of-fit of the model.  
Compare your value to the results of `statsmodels` from the previous exercise. Does the model fit well to the data?

We will now check if the assumptions of the linear regression model are satisfied. The assumptions can be summarized as the LINE rule:    

- **L**inear trend,
- **I**ndependent residuals,
- **N**ormally distributed residuals,
- **E**qual variance of residuals for all values of independent variables.

Check them visually by creating and analyzing the following diagnostic plots:   

- The residual value vs the fitted value
- The root square of the absolute value of standardized residuals vs the fitted value, 
- The histogram of residuals.

The first plot is used to check if the relationship between the response (the dependent variable) and the predictors (the independent variables) is linear, and for a very rough check if the residuals are uncorrelated. We expect values distributed symmetrically across the line $y=0$.  

The second plot is used to check homoskedasticity (equality for all values of the independent variables) of variance. We expect values distributed symmetrically across a straight horizontal line.    

The histogram is used to visualize the distribution of residuals.  
You can also use a qq-plot in this case if you know how to create and interpret it.

Finally, inspect either the influence plot or the leverage-resid2 plot,  [implemented in `statsmodels`](https://www.statsmodels.org/dev/examples/notebooks/generated/regression_plots.html).  
Both plots used to detect outliers that highly influence the model parameters. 


Decide which assumptions are satisfied to an appropriate degree.  
If you detect an outlying observation, remove it from the data set, run the calculations and diagnostics again and check if it improves the model fit.  

If you detect heteroskedasticity (non-constant variance of residuals), transforming the data may help.  
You may transform both the dependent and independent variable.  
Transforming the latter changes the functional relationship between the variables (i.e. whether they are linearly related), while transforming the former changes both the relationship and the structure of the residual variance.  

Estimate the average error in PLN that you would make if you used your model to predict the income of a municipality from its population.  

In [28]:
## Write your code here.  
residuals = y - results.fittedvalues
res_mean, res_sd = residuals.mean(), residuals.std()
stand_residuals = (residuals - res_mean) / res_sd
stand_residuals

0      -0.933176
1      -0.071538
2       0.456577
3      -0.227517
4       0.268326
          ...   
2504    0.119900
2505    0.275031
2506   -2.533633
2507   -6.738972
2508    0.860732
Length: 2477, dtype: float64

In [29]:
def coefficient_of_determination(residuals, y):
  SS_res = np.sum(np.square(residuals))
  y_mean = y.mean()
  SS_tot = np.sum(np.square(y - y_mean))
  return 1 - (SS_res / SS_tot)

In [31]:
coefficient_of_determination(residuals, y)

0.9393329661407305

In [37]:
df = pd.DataFrame()
df['prediction'] = y
df['residual'] = residuals

fig = px.scatter(
    df, x='prediction', y='residual', trendline='ols'
)
fig.show()

In [38]:
df['root_square_of_std_res'] = np.sqrt(np.abs(stand_residuals))

In [40]:
fig = px.scatter(
    df, x='prediction', y='root_square_of_std_res', trendline='ols')
fig.show()

In [42]:
fig = px.histogram(df, x='residual')
fig.show()

In [43]:
x = x[df['root_square_of_std_res'] < 1]
y = y[df['root_square_of_std_res'] < 1]

In [44]:
model = sm.OLS(y, x)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            2020_income   R-squared:                       0.860
Model:                            OLS   Adj. R-squared:                  0.859
Method:                 Least Squares   F-statistic:                     901.2
Date:                Tue, 11 Jan 2022   Prob (F-statistic):               0.00
Time:                        00:53:03   Log-Likelihood:                -42235.
No. Observations:                2357   AIC:                         8.450e+04
Df Residuals:                    2340   BIC:                         8.460e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
2020_popul

In [51]:
## Write your code here.  
residuals = y - results.fittedvalues
res_mean, res_sd = residuals.mean(), residuals.std()
stand_residuals = (residuals - res_mean) / res_sd
stand_residuals

0       2.250445
1       0.441102
2       0.049888
3       0.199665
4      -0.018985
          ...   
2502    0.339232
2503   -0.486574
2504   -0.157343
2505   -0.013978
2508    6.053102
Length: 2357, dtype: float64

In [46]:
df = pd.DataFrame()
df['prediction'] = y
df['residual'] = residuals

fig = px.scatter(
    df, x='prediction', y='residual', trendline='ols'
)
fig.show()

In [47]:
df['root_square_of_std_res'] = np.sqrt(np.abs(stand_residuals))

In [48]:
fig = px.scatter(
    df, x='prediction', y='root_square_of_std_res', trendline='ols')
fig.show()

In [49]:
fig = px.histogram(df, x='residual')
fig.show()

In [50]:
y_log = np.log1p(y)
model = sm.OLS(y_log, x)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            2020_income   R-squared:                       0.678
Model:                            OLS   Adj. R-squared:                  0.676
Method:                 Least Squares   F-statistic:                     307.8
Date:                Tue, 11 Jan 2022   Prob (F-statistic):               0.00
Time:                        00:58:07   Log-Likelihood:                -1542.1
No. Observations:                2357   AIC:                             3118.
Df Residuals:                    2340   BIC:                             3216.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
2020_popul

In [52]:
residuals = y_log - results.fittedvalues
res_mean, res_sd = residuals.mean(), residuals.std()
stand_residuals = (residuals - res_mean) / res_sd
stand_residuals

0      -0.704520
1       0.668724
2       1.119945
3       0.079099
4       0.608697
          ...   
2502    0.760448
2503   -1.577158
2504    0.139437
2505    0.836884
2508    0.879898
Length: 2357, dtype: float64

In [53]:
df = pd.DataFrame()
df['prediction'] = y_log
df['residual'] = residuals

fig = px.scatter(
    df, x='prediction', y='residual', trendline='ols'
)
fig.show()

In [55]:
df['root_square_of_std_res'] = np.sqrt(np.abs(stand_residuals))

In [56]:
fig = px.scatter(
    df, x='prediction', y='root_square_of_std_res', trendline='ols')
fig.show()

<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60">

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'>


Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego 
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej".   
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>